In [48]:
from pyspark import SparkConf, SparkContext

In [49]:
#creation of a python dictionary.Later the movie ID can be converted into movie name 
def loadMovieNames():
    movieNames = {}
    with open("/Users/hamsinisankaran/Downloads/ml-100k/u.item",encoding='ISO-8859-1') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [50]:
#take each line of u.data and convert into movieId and rating (1.0).This way we can add up all the ratings and divide 
#by the total ratings and find out the worst movie listings 
def parseInput(line):
    fields = line.split()
    return (int(fields[1]), (float(fields[2]), 1.0))

In [52]:
#main function 
if __name__ == "__main__":
    #The main script creates the sparkContext 
    conf = SparkConf().setAppName("WorstMovies")
    #sc = SparkContext(conf = conf)
    
#load up the movie ID and the 
movieNames = loadMovieNames()

#load up the raw movie data 
lines = sc.textFile("/Users/hamsinisankaran/Downloads/ml-100k/u.data")

#convert to movie id and rating 1.0 
movieRatings = lines.map(parseInput)

#reduce to movie ID, sumofRatings and total ratings 
ratingsTotalandCount = movieRatings.reduceByKey(lambda movie1, movie2: (movie1[0] + movie2[0], movie1[1] + movie2[1] ))

#map to movie ID and average rating 
averageRatings = ratingsTotalandCount.mapValues(lambda totalandCount : totalandCount[0]/ totalandCount[1])

#sort by average rating
sortedmovies = averageRatings.sortBy(lambda x : x[1])

#take the top 10 results
results = sortedmovies.take(10)

#print them out 
for result in results:
    print(movieNames[result[0]], result[1])


Amityville: Dollhouse (1996) 1.0
Somebody to Love (1994) 1.0
Every Other Weekend (1990) 1.0
Homage (1995) 1.0
3 Ninjas: High Noon At Mega Mountain (1998) 1.0
Bird of Prey (1996) 1.0
Power 98 (1995) 1.0
Beyond Bedlam (1993) 1.0
Falling in Love Again (1980) 1.0
T-Men (1947) 1.0
